In [24]:
from __future__ import unicode_literals, print_function
import pandas as pd
import sys
from spacy.lang.en import English
from featureExtract import extractFeatures

nlp = English()
nlp.add_pipe('sentencizer')

"""
python3 extract.py <doclist>

TEXT:filename identifier
ACQUIRED:entities that were acquired
ACQBUS:the business focus of the acquired entities
ACQLOC:the location of the acquired entities
DLRAMT:the amount paid for the acquired entities
PURCHASER:entities that purchased the acquired entities
SELLER:entities that sold the acquired entities
STATUS:status description of the acquisition event
"""

'\npython3 extract.py <doclist>\n\nTEXT:filename identifier\nACQUIRED:entities that were acquired\nACQBUS:the business focus of the acquired entities\nACQLOC:the location of the acquired entities\nDLRAMT:the amount paid for the acquired entities\nPURCHASER:entities that purchased the acquired entities\nSELLER:entities that sold the acquired entities\nSTATUS:status description of the acquisition event\n'

In [25]:
# read in training data
docFeats = ["path", "filename", "text"]

docsInfo = []
dataFolder = 'data/development-docs'
dataNames = os.listdir(dataFolder)
for dataName in dataNames:
    dcInf = []
    dcInf.append(dataFolder)
    dcInf.append(dataName)
    with open(os.path.join(dataFolder, dataName), 'r') as f:
        txt = ""
        for l in f.readlines():
            txt += l.strip()
        dcInf.append(txt)
    docsInfo.append(dcInf)
df = pd.DataFrame(docsInfo, columns=docFeats)
df["labelDict"] = ""


In [26]:
###Try linear regressor, multilabel with 7 labels
## use features from last project except vectorized (contextual vect hopefully) + doc number?
## words, POS tags, dependency relations, NER classes, semantic categories, and a context window- vectorized doc. 
## one hot encode output vector and sigmoid it

## train on corpus
# df.loc[df['filename'] == keyName.replace(".key", ""), 'labelDict'] = dic
df

,path,filename,text,labelDict
0,data/development-docs,10042,Meridian Energy Inc and CastoneDevelopment Cor...,
1,data/development-docs,10066,"Security Pacific Corp andUsers Inc, a credit u...",
2,data/development-docs,10158,Americanture Inc said it haspurchased American...,
3,data/development-docs,10171,Viacom International Inc said it setApril 6 as...,
4,data/development-docs,1029,Scott's Hospitality Inc said itacquired all is...,
...,...,...,...,...
393,data/development-docs,9733,British Petroleum Co Plc said itintended to m...,
394,data/development-docs,9809,Mickelberry Corp said it has completedthe prev...,
395,data/development-docs,982,H.J. Heinz chairman Tony O'Reillywould be int...,
396,data/development-docs,9858,MCO Holdings Inc said itsshareholders and thos...,


## Create Labels

Creates a an array dictionaries of {wd:label}

One array for each key file

In [27]:
outps = ['-', 'B-AQ', 'I-AQ', 'B-AQB', 'I-AQB', 'B-AQL', 'I-AQL', 'B-AMT', 'I-AMT', 'B-PURCH', 'I-PURCH', 'B-SELL', 'I-SELL', 'B-STAT', 'I-STAT']
cats = ['TXT','AQ', 'AQB', 'AQL', 'AMT', 'PURCH', 'SELL', 'STAT']
#Creaetes dictionary of wd:label
##TODO: go through and add to data

# Get list of paths
keyFolder = 'data/development-anskeys'
keyNames = os.listdir(keyFolder)
allDics = {}
for keyName in keyNames:
    dic = {}
    with open(os.path.join(keyFolder, keyName), 'r') as f:
        i = 0
        for l in f.readlines():
            wds = l.split()
            if i != 0:
                for j in range(len(wds)):
                    if j == 1: #beginning of phrase
                        pref = "B-"
                    else: #middle of phrase
                        pref = "I-"
                    if j != 0 and i < len(cats):
                        post = cats[i]
                        dic[wds[j].replace("\"", "")] = pref+post
            i+= 1
    allDics[keyName.replace(".key", "")] = (dic)




## Create Data

Creates list of feature vectors (per word)

In [32]:
import numpy as np
import random
#TODO: convert wds to embeddings and add context embedding. Also one hot label

# Create data
features = ["LABEL", "ABBR","CAP","LOC","POS","POS+1","POS-1","PREF","SUFF","WORD","WORD+1","WORD-1","TAG","TAG+1","TAG-1"]

trainData = []
testData = []
#split into sentences
for text, filename in zip(df['text'].values, df['filename'].values):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    test = False
    if random.random() <= 0.20: #put about 20& of articles in test set
        test = True
    for s in sentences:
        if test:
            testData += extractFeatures(s, allDics[filename])
        else:
            trainData += extractFeatures(s, allDics[filename])


print(np.shape(trainData))
print(np.shape(testData))
print(trainData[0])

(31244, 15)
(7938, 15)
['B-PURCH', 0, 1, 0, 'PROPN', 'PROPN', 'PHIPOS', 0, 0, 'Meridian', 'Energy', 'PHI', 'NNP', 'NNP', 'PHITAG']


In [33]:
trainDf = pd.DataFrame(trainData, columns=features)
trainDf.to_csv("trainData.csv", index=False)

testDf = pd.DataFrame(testData, columns=features)
testDf.to_csv("testData.csv", index=False)

## Run model trained on ^^ 

In [82]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from ml import vectorize_data, read_csv_for_ml, logistic

#For word in testData
dictvect = joblib.load("dicVec.joblib")
vect_test = dictvect.transform(testDf.to_dict('records'))

loaded_model = pickle.load(open('logistic_regression_model.joblib', 'rb'))

result = loaded_model.score(vect_test, test_labels)
print(result)

#save to dicts

0.8733938019652305


In [134]:
dictvect = joblib.load("dicVec.joblib")
loaded_model = pickle.load(open('logistic_regression_model.joblib', 'rb'))

ind = 0

X = testDf.drop('LABEL', axis=1)
test_vect = dictvect.transform(X.to_dict("records"))
pred = loaded_model.predict(vect_test)
print(pred[ind], dictvect.inverse_transform([vect_test[ind]]), testData[ind])

B-PURCH [{'CAP': 1.0, 'POS+1=PROPN': 1.0, 'POS-1=PHIPOS': 1.0, 'POS=PROPN': 1.0, 'TAG+1=NNP': 1.0, 'TAG-1=PHITAG': 1.0, 'TAG=NNP': 1.0, 'WORD+1=Pacific': 1.0, 'WORD-1=PHI': 1.0, 'WORD=Security': 1.0}] ['B-PURCH', 0, 1, 0, 'PROPN', 'PROPN', 'PHIPOS', 0, 0, 'Security', 'Pacific', 'PHI', 'NNP', 'NNP', 'PHITAG']


In [138]:
doclisttxt = 'doclist.txt'

#constants
dictvect = joblib.load("dicVec.joblib")
loaded_model = pickle.load(open('logistic_regression_model.joblib', 'rb'))
features = ["LABEL", "ABBR","CAP","LOC","POS","POS+1","POS-1","PREF","SUFF","WORD","WORD+1","WORD-1","TAG","TAG+1","TAG-1"]

## Add all docs to dataframe
docFeats = ["path", "filename", "text"]
doclist = open(doclisttxt, 'r')
docsInfo = []
for doc in doclist.readlines():
    dcInf = []
    pcs = doc.strip().split("/")
    dcInf.append("/".join(pcs[0:-1]))
    dcInf.append(pcs[-1])
    with open(doc.strip()) as txtFile:
        txt = ""
        for l in txtFile.readlines():
            txt += (" " + l.strip())
        dcInf.append(txt.strip())
    docsInfo.append(dcInf)
df = pd.DataFrame(docsInfo, columns=docFeats)


In [159]:
## iterate through text and perform interence
for text, filename in zip(df['text'].values, df['filename'].values):
    print(filename, text)
    outpDict = {'TXT': [filename] ,'AQ': [], 'AQB': [], 'AQL': [], 'AMT': [], 'PURCH': [], 'SELL': [], 'STAT': []}
    
    #extract features from text
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    data = []
    for s in sentences:
        data += extractFeatures(s, '-') #allDics[filename])
    data = np.array(data)

    #perform interence
    featDf = pd.DataFrame(data, columns=features)
    X = featDf.drop('LABEL', axis=1)
    print("featdf", X.shape)
    test_vect = dictvect.transform(X.to_dict("records"))
    pred = loaded_model.predict(test_vect)

    #Read through predictions add to dict
    for idx, p in enumerate(pred):
        postf = p.split("-")[-1]
        if postf in outpDict:
            outpDict[postf].append(X.iloc[idx]['WORD'])
        #else ignore it, probably a -

    #stringify predictions dict and add to dataframe? or just print to file
    outStr = ""
    for lab in outpDict.keys():
        outStr += (lab + ": \"")
        outStr += " ".join([wd for wd in outpDict[lab]])
        outStr += "\"\n"
    print(outStr)
    break


10042 Meridian Energy Inc and Castone Development Corp, a privately-held company, jointly announced that they have decided to terminate the letter of intent under which Meridian would have acquired Castone.
featdf (29, 14)
TXT: 10042
AQ: 
AQB: 
AQL: 
AMT: 
PURCH: Inc
SELL: 
STAT: of intent



In [156]:
outpDict.keys

<function dict.keys>

In [136]:
featDf

,ABBR,CAP,LOC,POS,POS+1,POS-1,PREF,SUFF,WORD,WORD+1,WORD-1,TAG,TAG+1,TAG-1
0,0,1,0,PROPN,PROPN,PHIPOS,0,0,Gulf,Applied,PHI,NNP,NNP,PHITAG
1,0,1,0,PROPN,PROPN,PROPN,0,0,Applied,Technologies,Gulf,NNP,NNPS,NNP
2,0,1,0,PROPN,PROPN,PROPN,0,1,Technologies,Inc,Applied,NNPS,NNP,NNP
3,0,1,0,PROPN,VERB,PROPN,0,0,Inc,said,Technologies,NNP,VBD,NNPS
4,0,0,0,VERB,PRON,PROPN,0,0,said,it,Inc,VBD,PRP,NNP
5,0,0,0,PRON,VERB,VERB,0,0,it,sold,said,PRP,VBD,VBD
6,0,0,0,VERB,PRON,PRON,0,0,sold,its,it,VBD,PRP$,PRP
7,0,0,0,PRON,NOUN,VERB,0,0,its,subsidiaries,sold,PRP$,NNS,VBD
8,0,0,0,NOUN,VERB,PRON,0,0,subsidiaries,engaged,its,NNS,VBN,PRP$
9,0,0,0,VERB,ADP,NOUN,0,0,engaged,in,subsidiaries,VBN,IN,NNS
